### Recipes Convert

This notebook explores converting a preprocessed recipe file into the satisfactory tools format. 

---

Overview of that format:

1. Recipes - the thing we're going for the most. 
    * slug + name
    * className - a unique recipe ID that seems to be used like a slug. Not sure why it's not just the slug.
    * alternate - we'll need to come up with this, whether this is the primary way for getting a resource or not. Informs which recipes are used by default
    * time - time to craft (very relevant)
    * manualTimeMultiplier - used in Satisfactory, doesn't matter for greg but we'll need to include a placeholder.
    * ingredients - a list of ingredients, each with a className (NOT slug) and amount
    * forBuilding - I don't think this is actually used in SatisfactoryTools? The buildings tab is built from the buildings list at the root level.
    * inMachine + inHand + inWorkshop - can this recipe be done by x? Affect what info is shown in the UI. We'll need to come up with this
    * products - a list of products, each with a className (NOT slug) and amount
    * isVariablePower - used in Satisfactory, I don't think it matters for greg but we'll need to include a placeholder. If true,  we'll need to specify minPower and maxPower, the UI will show the "Average" as the average of those two.
    * producedIn - if it can be made inMachine, what machine(s)? uses className. Machines inform power use calculations so we'll need to be smart about this one.
2. Items
    * slug + name + className
    * stackSize - how many can be in a stack
    * liquid
    * radioactivity - not sure if we dump/have access to this information for greg items
    * sinkPoints - coming up with a value for this would help inform default recipes, actually.
    * fluidColor - we throw this data away rn, actually. We could use it to inform the UI.
    * fuelValue - relevant to greg but I explicitly ignore fuel values rn
3. Schematics - research exists in Satisfactory. We'll leave this blank.
4. Generators - technically relevant to greg. The UI has references to this data but I think the UI components for that code aren't turned on/included in the website. Leave blank and hopefully nothing breaks.
5. Resources - seems to define what can be mined. This will be a tough one to fill in as we'll have to determine what users will want to have an inputs in the normal case. There will also be *so many of them* that the UI will prolly struggle.
    * item className - corresponds to listing in Items
    * pingColor - relevant to the game but not SatistactoryTools and not greg
    * speed - mining speed I think? It's 1 for every Satisfactory resource.
6. Miners - only seems to be used in the UI to determine whether something is a manufacturer? Try leaving it blank.
7. Buildings - important field, lists machines and power consumption.
    * slug + name + className
    * metadata - stuff about power. powerConsumption is quite relevant, but powerConsumptionExponent seems to be overriden in the UI to 1.6, so it doesn't matter. The code suggests that there should be a way to manually set the overclock, but I can't seem to find it. We'll assume no overclocking for this first version. I'll need to edit the UI to not calculate overclocks too, becuase the rules are different for greg vs. Satisfactory.
    * size - doesn't seem to be used in the UI, set to 0.
    * buildMenuPriority - doesn't seem to be used in the UI, set to 0.
    * category - not used in the UI, leave blank.





# Stuff to figure out
---

- [ ] classNames for everything. The Desc_ / Build_ / BP_ / ResourceSink_ / etc. prefixes appear in the codebase but so far as I can tell are only used to gather information for the codex. I think using Desc_ for all items and Build_ for all buildings will satiate the UI, but I'm not sure. It might reduce file size to omit them.
- [ ] Nice user-facing names for everything. This is a big one, especially for recipes, as users will need to be able to quicky filter through a lot of them.
- [ ] A smart filter for what we include in the output file! We don't want to include every single item in the game, just the ones that are relevant to the user. The UI is going to be hella loaded down as it is.
- [ ] Machines + their power consumption. The same class of machine in greg can have many different tier-based power consumptions. As a hack, we'll add a different machine for every tier. This will take some work.
- [ ] Determine what is an input resource. Probably will be based on an oredict rule.
- [ ] How to determine the primary and alternate recipes for a resource. This is a big one - maybe the biggest - because starting with reasonable defaults for each item will make life much easier for the user, but it will require not-obvious logic to determine what is more expensive/higher tier to make.
- [ ] Sink Value - the Satisfactory meaning doesn't exist in greg, but it might still be useful to calculate as an intermediary to selecting primary recipes.
- [ ] There are a bunch of constants in the UI that seem to inform the UI, like names for water, radioactive waste, etc. We can try omitting them to start.
- [ ] Descriptions for items in the UI. Certainly not necessary to start, but including some metadata we've got lying about, like source mod
- [ ] Other things...?


In [3]:
from tools.nerd_format import RecipeFile
from tools.dump_format import RecipeStacks
from tools.util import parse_json, load_config

config = load_config()
recipes = parse_json("data/recipes_filtered.json", RecipeFile)
stacks = parse_json("data/recipes_stacks.json", RecipeStacks)

In [27]:
# Lets start filling out formatted data! Start with items
from tools.dump_format import GTFluid
from tools.st_format import Item
import numpy as np

def get_name(slug):
    if slug in stacks.items:
        return stacks.items[slug].displayName
    if slug in stacks.fluids:
        fluid = stacks.fluids[slug]
        return fluid.localizedName if isinstance(fluid, GTFluid) else fluid.fluidName
    return "Unknown"

# There are some items we never reference in a recipe. To reduce output sizes somewhat, we'll filter them out.
recipe_slugs = set()
for recipe in recipes.recipes:
    recipe_slugs.update(i.slug for i in recipe.inputs + recipe.outputs)
print("total:", len(stacks.items)+len(stacks.fluids), "used in recipes:", len(recipe_slugs))
known_slugs = set(stacks.items.keys()) | set(stacks.fluids.keys())
missing = recipe_slugs - known_slugs
print("Items missing from stacks:", len(missing), "Examples:", np.random.choice(list(missing), 5))
used_items = known_slugs & recipe_slugs
print("Items used in recipes:", len(used_items))



total: 30577 used in recipes: 25082
Items missing from stacks: 922 Examples: ['4350d12990' '4350d3926' '4350d4512' '4350d3368' '4350d3905']
Items used in recipes: 24160


In [28]:
[get_name(item) for item in used_items]

['Impure Pile of Sapphire Dust',
 'Nitinol 60 Rotor',
 'Bismuth Turbine Blade',
 'Tantalum Carbide Bolt',
 'Green Sapphire Ore',
 'Smooth Black Granite',
 'Crushed Vulcanite Ore',
 '16x Cobalt Wire',
 'Block of Bismuth',
 'Tiny Pile of Americium Dust',
 'Coil Wire [LuV]',
 'Tanzanite Ore',
 'Double Molybdenum Ingot',
 'SpaceTime Frame Box',
 'Pig Iron Shovel Head',
 'Raw Tengam Ore',
 'Adamantium Ore',
 'Chrome Ore',
 'Small Tellurium Spring',
 'Pikyonium 64B Ingot',
 'Orichalcum Arrow Head',
 'Purified Pile of Platinum Dust',
 'Crushed Huebnerite Ore',
 'Quintuple Ichorium Plate',
 'Livingwood Foil',
 'Arceus Alloy 2B Rod',
 'Transmission Component (ZPM)',
 '30k Coolant Cell',
 'Banded Iron Ore',
 'Long Superconductor Base HV Rod',
 'Ruthenium Ingot',
 'Terraformer',
 'Molten Magnetic Samarium Capsule',
 'ME Dense Smart Cable - Magenta',
 'Methanol Cell',
 'Purified Irarsite Dust',
 'Extruder Shape (Gear)',
 'Small Pile of Sodium Methoxide Dust',
 'Quadruple Carbon Plate',
 'Advanced 